1) Анализ временных рядов помесячных продаж (с 2006 г.) б/у и новых автомобилей.
Построить ARIMA и ECM, показатели взять: общий уровень цен, курс валюты, ВВП (лучше не его), уровень доходов.
Лучше, если по регионам.

http://greenway.icnet.ru/cars-sales-actual-russia.html - useful material
https://www.autostat.ru/press-releases/47248/ - top 30 regions by ...

Рынок легковых автомобилей с пробегом в сентябре 2022 года
За дополнительной информацией обращайтесь в агентство «АВТОСТАТ» к Азату Тимерханову
по телефонам: +7 (8482) 95-64-36, +7 (499) 685-01-51, или по e-mail: press@autostat.ru

https://www.autostat.ru/research/product/322/ - RADAR

In [277]:
import pandas as pd

demand_df = pd.read_excel('data_time_series/demand_for_new_cars.xlsx')  # https://auto.vercity.ru/statistics/sales/europe/russia/

demand_df = pd.melt(demand_df, id_vars=['Year'])\
    .rename({'variable': 'Month', 'value': 'demand_new'}, axis=1)
demand_df['Month'] = pd.to_datetime(demand_df['Month'], format='%B').dt.month
demand_df['date'] = pd.to_datetime(demand_df[['Year', 'Month']].assign(DAY=1))
demand_df = demand_df[['date', 'demand_new']].sort_values(['date']).reset_index(drop=True)

# print(demand_df.tail(3))
demand_df = demand_df[:-1]

demand_df['demand_new'] = demand_df['demand_new'].apply(lambda x: x.replace(' ', '')).astype('int')

demand_df.tail()

,date,demand_new
198,2022-07-01,36605
199,2022-08-01,47090
200,2022-09-01,52738
201,2022-10-01,45228
202,2022-11-01,46403


In [278]:
# Индексы потребительских цен на непродовольственные товары по Российской Федерации в 1991-2022гг.
inf_df = pd.read_excel('data_time_series/inflation.xlsx')  # https://rosstat.gov.ru/statistics/price

inf_df = pd.melt(inf_df, id_vars=['month'])\
    .rename({'variable': 'year', 'value': 'inflation_rate'}, axis=1)
inf_df['date'] = pd.to_datetime(inf_df[['year', 'month']].assign(DAY=1))

inf_df = inf_df[['date', 'inflation_rate']].sort_values(['date']).reset_index(drop=True)

# print(inf_df.tail(3))
inf_df = inf_df[:-1]

inf_df['inflation_rate'] = inf_df['inflation_rate'].astype('float')

inf_df.tail()

,date,inflation_rate
378,2022-07-01,99.56
379,2022-08-01,99.95
380,2022-09-01,100.15
381,2022-10-01,100.04
382,2022-11-01,100.06


In [279]:
brent_df = pd.read_csv('data_time_series/ICE.BRN_050101_230109.txt')     # https://www.finam.ru/profile/tovary/brent/export/

brent_df = brent_df[['<DATE>', '<CLOSE>']].rename({'<DATE>': 'date', '<CLOSE>': 'brent_close_price'}, axis=1)
brent_df['date'] = pd.to_datetime(brent_df['date'], format='%Y%m%d')

brent_df.tail()

,date,brent_close_price
212,2022-09-01,85.57
213,2022-10-01,92.85
214,2022-11-01,86.64
215,2022-12-01,85.97
216,2023-01-01,79.45


In [280]:
pp_df = pd.read_excel('data_time_series/purchasing_power.xlsx')     # https://www.fedstat.ru/indicator/31326

pp_df.columns = ['year', 'Product', 'Russia']

pp_df['year'] = pp_df['year'].fillna(method='ffill').astype('int')
pp_df = pp_df[pp_df['year'] >= 2006]
pp_df = pp_df[pp_df['Product'] != 'Бензин']

pp_df = pp_df.pivot(index='year', columns='Product', values='Russia')
pp_df = pp_df.reset_index(drop=False)
pp_df.columns = ['year', 'pp_petrol_92', 'pp_petrol_95', 'pp_city_bus', 'pp_municipal_bus']

print(f"Correlation between petrol 92 and petrol 95: {pp_df[['pp_petrol_92', 'pp_petrol_95']].corr().iloc[0, 1].round(4)}")

pp_df = pp_df[['year', 'pp_petrol_95']]

def df_year_to_months(df_input):
    df = pd.concat([df_input for _ in range(12)]).sort_values('year').reset_index(drop=True)
    df['year'] = pd.to_datetime(df['year'], format='%Y')
    df['year'] = df.apply(lambda x: x['year'] + pd.DateOffset(months=x.name % 12), axis=1)
    df = df.rename({'year': 'date'}, axis=1)
    return df

pp_df = df_year_to_months(pp_df)

pp_df.tail()

Correlation between petrol 92 and petrol 95: 0.9985


,date,pp_petrol_95
187,2021-08-01,818.4
188,2021-09-01,818.4
189,2021-10-01,818.4
190,2021-11-01,818.4
191,2021-12-01,818.4


In [281]:
income_df = pd.read_excel('data_time_series/nominal_income.xlsx')   # https://www.fedstat.ru/indicator/30992

income_df = income_df[income_df['Region'] == 'Российская Федерация']
income_df = income_df.T.reset_index()
income_df.columns = ['year', 'nominal_income']

income_df = income_df.drop(0, axis=0).reset_index(drop=True)

income_df = df_year_to_months(income_df)

income_df.tail()

,date,nominal_income
319,2021-08-01,40272.0
320,2021-09-01,40272.0
321,2021-10-01,40272.0
322,2021-11-01,40272.0
323,2021-12-01,40272.0


In [282]:
# HAVEN'T USED

usdrub_df = pd.read_csv('data_time_series/usdrub_m.csv')    # https://stooq.com/q/d/?s=usdrub&c=0&o=1111111&i=m&o_s=1&o_d=1&o_p=1&o_n=1&o_o=1&o_m=1&o_x=1

usdrub_df = usdrub_df[['Date', 'Close']].rename({'Date': 'date', 'Close': 'usdrub_close_price'}, axis=1)

usdrub_df['date'] = pd.DatetimeIndex(usdrub_df['date']) + pd.DateOffset(1)

usdrub_df.tail()

,date,usdrub_close_price
323,2022-10-01,60.75
324,2022-11-01,61.70
325,2022-12-01,60.90
326,2023-01-01,73.20
327,2023-02-01,72.77


In [291]:
# Real Broad Effective Exchange Rate for Russia
rbrubis_df = pd.read_csv('data_time_series/RBRUBIS.csv')    # https://fred.stlouisfed.org/series/RBRUBIS
rbrubis_df = rbrubis_df.rename({'DATE': 'date', 'RBRUBIS': 'rbrubis'}, axis=1)
rbrubis_df['date'] = pd.to_datetime(rbrubis_df['date'])

rbrubis_df.tail()

,date,rbrubis
342,2022-07-01,115.52
343,2022-08-01,110.73
344,2022-09-01,113.90
345,2022-10-01,112.43
346,2022-11-01,110.31


In [292]:
indicators_df = pd.read_excel('data_time_series/indicators.xlsx')      # https://rosstat.gov.ru/folder/210/document/13396

indicators_df = indicators_df.T.reset_index()
indicators_df.columns = ['year', 'real_income', 'real_wage', 'real_pension', 'Gini']
indicators_df = indicators_df.drop(0, axis=0).reset_index(drop=True)

indicators_df = df_year_to_months(indicators_df)

indicators_df.tail()

,date,real_income,real_wage,real_pension,Gini
367,2021-08-01,103.2,104.5,104.1,0.409
368,2021-09-01,103.2,104.5,104.1,0.409
369,2021-10-01,103.2,104.5,104.1,0.409
370,2021-11-01,103.2,104.5,104.1,0.409
371,2021-12-01,103.2,104.5,104.1,0.409


In [294]:
all_df = demand_df
other_dfs = [inf_df, brent_df, pp_df, income_df, rbrubis_df, indicators_df]

for df_i in other_dfs:
    all_df = pd.merge(all_df, df_i)

all_df

,date,demand_new,inflation_rate,brent_close_price,pp_petrol_95,nominal_income,rbrubis,real_income,real_wage,real_pension,Gini
0,2006-01-01,95253,100.41,65.43,533.0,10154.8,84.78,113.466448,113.3,105.1,0.415
1,2006-02-01,116309,100.45,60.05,533.0,10154.8,87.08,113.466448,113.3,105.1,0.415
2,2006-03-01,147308,100.42,64.94,533.0,10154.8,88.40,113.466448,113.3,105.1,0.415
3,2006-04-01,169045,100.28,72.00,533.0,10154.8,88.03,113.466448,113.3,105.1,0.415
4,2006-05-01,165121,100.41,69.00,533.0,10154.8,87.34,113.466448,113.3,105.1,0.415
...,...,...,...,...,...,...,...,...,...,...,...
187,2021-08-01,121329,100.80,71.70,818.4,40272.0,76.33,103.2,104.5,104.1,0.409
188,2021-09-01,127025,100.64,78.36,818.4,40272.0,77.24,103.2,104.5,104.1,0.409
189,2021-10-01,133256,100.78,83.25,818.4,40272.0,79.98,103.2,104.5,104.1,0.409
190,2021-11-01,132474,100.72,71.15,818.4,40272.0,79.65,103.2,104.5,104.1,0.409
